In [1]:
import cv2
import PIL.Image
from matplotlib import pyplot as plt
import ipywidgets
from IPython.display import display

import json

import torch
import torchvision.transforms as transforms

from torch2trt import TRTModule, torch2trt
import trt_pose.models
import trt_pose.coco
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

from jetcam.usb_camera import USBCamera
from jetcam.utils import bgr8_to_jpeg

In [2]:
# config
WIDTH = 224
HEIGHT = 224
UNOPTIMIZED_MODEL = r'resnet18_baseline_att_224x224_A_epoch_249.pth'
OPTIMIZED_MODEL = r'resnet18_baseline_att_224x224_A_epoch_249_trt.pth'
HUMAN_CONFIG_DIR = r'human_pose.json'

In [12]:
def get_topology():
    with open(HUMAN_CONFIG_DIR, 'r') as f:
        human_pose = json.load(f)
    return human_pose, trt_pose.coco.coco_category_to_topology(human_pose)

def optimize_model(human_pose):
    num_parts = len(human_pose['keypoints'])
    num_links = len(human_pose['skeleton'])
    model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()
    model.load_state_dict(torch.load(UNOPTIMIZED_MODEL))
    data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()
    model_trt = torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)
    torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)

def get_optimized_model():
    model_trt = TRTModule()
    model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))
    return model_trt

def preprocess(image):
    mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
    std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def execute(change):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    draw_objects(image, counts, objects, peaks)
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    display(image_w)
    
def peek_a_frame(camera):
    execute({'new': camera.value})
    
def start_peek_frames(camera):
    camera.observe(execute, names='value')
    
def stop_peek_frames(camera):
    camera.unobserve_all()

In [4]:
human_pose, topology = get_topology()
parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

In [15]:
# process and save a optimized model
# optimize_model(human_pose)

In [5]:
# load model
model_trt = get_optimized_model()

In [6]:
# init display
image_w = ipywidgets.Image(format='jpeg')

In [7]:
camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30)
camera.running = True

In [13]:
camera.running = True

In [14]:
peek_a_frame(camera)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [27]:
start_peek_frames(camera)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

...

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [11]:
stop_peek_frames()
camera.running = False